# COMP30027 Machine Learning Project 2

## Imports

In [1]:
import math
import numpy as np
import pandas as pd

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import BernoulliRBM, MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

## Load Datasets

In [2]:
train_raw = pd.read_csv("./Data/train_raw.csv", header=None, na_values='?', keep_default_na=False, usecols=[0, 2, 6])
train_top10 = pd.read_csv("./Data/train_top10.csv", header=None, na_values='?', keep_default_na=False)

dev_raw = pd.read_csv("./Data/dev_raw.csv", header=None, na_values='?', keep_default_na=False, usecols=[0, 2, 6])
dev_top10 = pd.read_csv("./Data/dev_top10.csv", header=None, na_values='?', keep_default_na=False)

test_raw = pd.read_csv("./Data/test_raw.csv", header=None, na_values='?', keep_default_na=False, usecols=[0, 2, 6])
test_top10 = pd.read_csv("./Data/test_top10.csv", header=None, na_values='?', keep_default_na=False)

## Train (Initial Test)
### k-NN

In [3]:
# cf1 = KNeighborsClassifier(int(math.sqrt(train_top10.size)), "distance", n_jobs=-1)
# cf1.fit(train_top10[:, 1:31], train_top10[:, 31])
# cf1.fit(train_top10[:, 1:31], train_top10[:, 31])
# t = cf1.predict(dev_top10[:, 1:31]) == dev_top10[:, 31]
# t = np.bincount(t)
# s = t[1] / (t[0] + t[1])
# print(f"score: {s}")

### Gaussian Naive Bayes

In [4]:
# cf2 = GaussianNB()
# cf2.fit(train_top10[:, 1:31], train_top10[:, 31])
# t = cf2.predict(dev_top10[:, 1:31]) == dev_top10[:, 31]
# t = np.bincount(t)
# s = t[1] / (t[0] + t[1])
# print(f"score: {s}")

### Random Decision Forest Classifier

In [5]:
# cf3 = RandomForestClassifier()
# cf3.fit(train_top10[:, 1:31], train_top10[:, 31])
# t = cf3.predict(dev_top10[:, 1:31]) == dev_top10[:, 31]
# t = np.bincount(t)
# s = t[1] / (t[0] + t[1])
# print(f"score: {s}")

### Multilayer Perceptron Classifier

In [6]:
# cf4 = MLPClassifier()
# cf4.fit(train_top10[:, 1:31], train_top10[:, 31])
# t = cf4.predict(dev_top10[:, 1:31]) == dev_top10[:, 31]
# t = np.bincount(t)
# s = t[1] / (t[0] + t[1])
# print(f"score: {s}")

### Ada Boost Classifier with Decision Tree _(default)_

In [7]:
# cf5 = AdaBoostClassifier(n_estimators=100)
# cf5.fit(train_top10[:, 1:31], train_top10[:, 31])
# t = cf5.predict(dev_top10[:, 1:31]) == dev_top10[:, 31]
# t = np.bincount(t)
# s = t[1] / (t[0] + t[1])
# print(f"score: {s}")

## Train (Combine UID)

First combine all UIDs

In [8]:
def unique(top10, raw):
    b = top10.copy()
    b[0] = raw[0]
    a = b.drop_duplicates(0)
    a.update(b.iloc[:, :31].groupby(0).mean())
    return a

a = unique(train_top10, train_raw)
b = unique(dev_top10, dev_raw)

c:\users\laitingsheng\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:4290: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[col] = expressions.where(mask, this, that)


### Gradient Boosting Classifier

In [9]:
cf1 = GradientBoostingClassifier(n_estimators=1000)
cf1.fit(a.values[:, 1:31], a.values[:, 31])

mapping = pd.Series(cf1.predict(b.values[:, 1:31]), b.iloc[:, 0])
predict_dev_top10 = dev_top10.copy()
predict_dev_top10[0] = dev_raw[0]

p = predict_dev_top10[[0, 31]]
def rc(row):
    row[31] = mapping[row[0]]
    return row
p = p.apply(rc, 1, raw=True)
p[0] = dev_top10[0]
predict_dev_top10.update(p)
re = (predict_dev_top10[31] == dev_top10[31]).value_counts(True)
print(re)

False    0.588194
True     0.411806
Name: 31, dtype: float64


### Multilayer Perceptron Classifier

In [10]:
cf2 = MLPClassifier()
cf2.fit(a.values[:, 1:31], a.values[:, 31])

mapping = pd.Series(cf2.predict(b.values[:, 1:31]), b.iloc[:, 0])
predict_dev_top10 = dev_top10.copy()
predict_dev_top10[0] = dev_raw[0]

p = predict_dev_top10[[0, 31]]
def rc(row):
    row[31] = mapping[row[0]]
    return row
p = p.apply(rc, 1, raw=True)
p[0] = dev_top10[0]
predict_dev_top10.update(p)
re = (predict_dev_top10[31] == dev_top10[31]).value_counts(True)
print(re)

False    0.57712
True     0.42288
Name: 31, dtype: float64


### Ada Boost Classifier with Decision Tree _(default)_

In [11]:
cf3 = AdaBoostClassifier(n_estimators=100)
cf3.fit(a.values[:, 1:31], a.values[:, 31])

mapping = pd.Series(cf3.predict(b.values[:, 1:31]), b.iloc[:, 0])
predict_dev_top10 = dev_top10.copy()
predict_dev_top10[0] = dev_raw[0]

p = predict_dev_top10[[0, 31]]
def rc(row):
    row[31] = mapping[row[0]]
    return row
p = p.apply(rc, 1, raw=True)
p[0] = dev_top10[0]
predict_dev_top10.update(p)
re = (predict_dev_top10[31] == dev_top10[31]).value_counts(True)
print(re)

False    0.588083
True     0.411917
Name: 31, dtype: float64


### Ensembled _(Stacking)_

In [30]:
enc = LabelEncoder()
enc.fit(a.values[:, 31])

na = pd.DataFrame()
na[0] = a[0]
na[1] = enc.transform(cf1.predict(a.values[:, 1:31]))
na[2] = enc.transform(cf2.predict(a.values[:, 1:31]))
na[3] = enc.transform(cf3.predict(a.values[:, 1:31]))
na[4] = a[31]
nb = pd.DataFrame()
nb[0] = b[0]
nb[1] = enc.transform(cf1.predict(b.values[:, 1:31]))
nb[2] = enc.transform(cf2.predict(b.values[:, 1:31]))
nb[3] = enc.transform(cf3.predict(b.values[:, 1:31]))
nb[4] = b[31]

ecf = AdaBoostClassifier(n_estimators=100)
ecf.fit(na.values[:, 1:4], na.values[:, 4])

mapping = pd.Series(ecf.predict(nb.values[:, 1:4]), nb.iloc[:, 0])
t = dev_top10.copy()
t[0] = dev_raw[0]

p = t[[0, 31]]
def rc(row):
    row[31] = mapping[row[0]]
    return row
p = p.apply(rc, 1, raw=True)
p[0] = dev_top10[0]
t.update(p)
re = (t[31] == dev_top10[31]).value_counts(True)
print(re)

False    0.588194
True     0.411806
Name: 31, dtype: float64


In [23]:
# cf = MLPClassifier()
# cf.fit(a.values[:, 1:31], a.values[:, 31])

# mapping = pd.Series(cf.predict(b.values[:, 1:31]), b.iloc[:, 0])
# predict_test_top10 = test_top10.copy()
# predict_test_top10[0] = test_raw[0]

# p = predict_test_top10[[0, 31]]
# def rc(row):
#     row[31] = mapping[row[0]]
#     return row
# p = p.apply(rc, 1, raw=True)
# p[0] = test_top10[0]
# predict_test_top10.update(p)
# re = predict_test_top10[[0, 31]]
# re.columns = ["Id", "Prediction"]
# re.to_csv("./Data/prediction.csv", index=False)